# Team (◕‿◕✿)'s Project Process

## 1. Scraping and cleaning our data

We will scrape and clean our data with the following steps:

1. Filter to AskReddit threads made in the last year.
2. Get the thread IDs of the search results from step (1).
3. Use praw to scrape parent comments from thread X.

### 1.1 Filter using reddit search

We want to get a decent sample size over a year -- we thus split a year's worth of data (11/01/2014 - 10/31/2015) into four parts of three months each. We then convert each of these time ranges into epochs, and format the range to use in URLs.

In [ ]:
import time

time_ranges = [('11-01-2014', '01-31-2015'), ('02-01-2015', '04-30-2015'), ('05-01-2015', '07-31-2015'), ('08-01-2015', '10-31-2015')] 
f_epoch = lambda x: int(time.mktime(time.strptime(x, '%m-%d-%Y')))
time_ranges = [str(f_epoch(start)) + '..' + str(f_epoch(end)) for (start, end) in time_ranges]
print time_ranges

For each element in ```time_ranges```, we create a url to filter to AskReddit threads in that time range. The URL for the filtering is:

https://www.reddit.com/r/AskReddit/search?sort=comments&q=timestamp%3A[time_range]&restrict_sr=on&syntax=cloudsearch

Breaking down the above URL:
* **/r/AskReddit**: filter to subreddit AskReddit
* **sort=comments**: the search results will be sorted based on number of comments
* **timestamp%3A1410739200..1411171200&**: restrict to 10/01/2014 00:00:00 to 10/01/2015 00:00:00, which are translated into epochs in the URL.
* **syntax=cloudsearch**: reddit's search is infamously bad. One reason is that its regular search syntax, Lucene, doesn't allow for timestamp search. We thus choose CloudSearch as our syntax instead, which allows us to use features (like timestamp searches) that Lucene does not allow.

reddit will return at most 1000 results for our search. 

###1.2 Get thread IDs

Now, we want to get all the search result URLs from our search. This sounds like a job for Beautiful Soup! First, using Developer Tools, we find our search contents:

![Image of HTML searching](images/part1_1.png)
![Image of HTML searching (2)](images/part1_2.png)

We next parse the HTML using Beautiful Soup to get all the thread URLs for the search results. After getting the thread URLs, we append '.json' to the end of the URL, and using regex to extract out the thread ID from the resulting json. For each time range query (4 in total), we return 25 threads for a total of 100 threads, and thus the resulting ```thread_ids``` has 100 strings.

(Note: the for-loop below can be written more nicely if reddit servers could handle more load! However, there are still HTTP error 429s after sleeping for 10 seconds. We circumvent this issue by checking how many URLs in ```url_list``` we manage to parse before hitting error 429, and continue appending from there.)

In [66]:
from bs4 import BeautifulSoup
import urllib2
import re

url_str1 = "https://www.reddit.com/r/AskReddit/search?sort=comments&q=timestamp%3A"
url_str2 = "&restrict_sr=on&syntax=cloudsearch"
url_list = []
for tr in time_ranges:
    time.sleep(30) # avoid HTTP request error
    url = url_str1 + tr + url_str2
    data = BeautifulSoup(urllib2.urlopen(url).read(), 'html.parser')
    rows = data.findAll("a", attrs = {'class': 'search-comments may-blank'}, limit=None)
    url_list += [str(r.attrs['href'])[:-17] + '.json' for r in rows]

thread_ids = []
for elt in url_list:
    time.sleep(10)
    json_text = urllib2.urlopen(elt).read()
    thread_ids.append(re.search(r'\"id\": \"([a-zA-Z0-9_]+)\"', json_text).group(1))

100 100


### 1.3 Scrape comments using praw

praw (Python Reddit API Wrapper), given the thread ID, will scrape all the parent comments from the thread. We will store the scraped comments in a dictionary: the keys are the thread ID, and the values are the list of parent comments in that thread. 

Unfortunately, the call ```replace_more_comments``` below takes up a lot of time, due to reddit's API request limits. It is possible to pass ```limit=None``` in ```replace_more_comments``` to get all the comments in the thread, but that often leads to HTTP error 429. We thus cap the number of requests at 100. The argument ```threshold``` checks how many replies the comment receives. We don't mind if our parent comment receives no replies, so we set ```threshold=0```.

In [92]:
import praw
r = praw.Reddit('Getting comments for CS109 Final Project')

thread_d = {}
for t_id in thread_ids:
    time.sleep(10) # lots of HTTP request errors!
    submission = r.get_submission(submission_id=t_id)
    submission.replace_more_comments(limit=100, threshold=0)
    all_comments = submission.comments
    thread_d[t_id] = all_comments

100
164673


Phew! We don't want to go through that again, so let's pickle our dictionary into the file ```all_comments_dict.p```, so we have it ready-to-go for the next run. Note that ```all_comments_dict.p``` is too big to upload to Github, so we had to zip the file ( ```all_comments_dict.p.zip```) in order to upload it. 

In [93]:
import cPickle
cPickle.dump(thread_d, open('all_comments_dict.p', 'wb')) 
thread_d_load = cPickle.load(open('all_comments_dict.p', 'rb'))
print len(thread_d_load)


100


###What elements are interesting?

We can look at the attributes of a PRAW comment and see what might be useful.


In [99]:
from pprint import pprint
pprint (vars(thread_d_load['2rb0pa'][0]))

{'_has_fetched': True,
 '_has_fetched_replies': True,
 '_info_url': u'https://api.reddit.com/api/info/',
 '_replies': [<praw.objects.Comment object at 0x210966f50>],
 '_submission': <praw.objects.Submission object at 0x21092d6d0>,
 '_underscore_names': [u'replies'],
 '_uniq': None,
 'approved_by': None,
 'archived': True,
 'author': Redditor(user_name='finndre'),
 'author_flair_css_class': None,
 'author_flair_text': None,
 'banned_by': None,
 'body': u'Private prisons with lockup quotas. States are contractually obligated to fill prison beds. wtf',
 'body_html': u'<div class="md"><p>Private prisons with lockup quotas. States are contractually obligated to fill prison beds. wtf</p>\n</div>',
 'controversiality': 0,
 'created': 1420430099.0,
 'created_utc': 1420401299.0,
 'distinguished': None,
 'downs': 0,
 'edited': False,
 'gilded': 0,
 'id': u'cnecm22',
 'json_dict': None,
 'likes': None,
 'link_id': u't3_2rb0pa',
 'mod_reports': [],
 'name': u't1_cnecm22',
 'num_reports': None,
 'p

At first glance, the interesting attributes are:
* the gold count
* number of upvotes
* time posted
* comment body
* author